# Mittelalterliche astronomische Diagramm Datenbank

Die Datenbank ist im  Datenformat JSON  in Repositorium der Edition Topoi publiziert.  Aus ihr wurde eine  flache Liste aller Diagramme  generiert, die über das Internet aus einem Git-Repositorium eingeladen wird.  Die Datenbank *diaattr.json* enthält als Forschungsdatenobjekt die Beschreibung der Diagramme mit Attributen, wie die Charakteristiken  der  Abbildungen und Manuskripte,  die Bibliothek des heutigen Halters  sowie die Stelle der Abbildung im Manuskript  sowie ihr Diagrammtyp.



In [ ]:
import pandas as pd
import medDiaJson as medDia
import roman_date as roda

In [ ]:
url = 'https://raw.githubusercontent.com/grasshoff/ComputationalHistoryScience/master/diaattr.json'
diagrams = pd.read_json(url)
diagrams.head(1)

In [ ]:
len(diagrams)

In [ ]:
diagrams.keys()

 Insgesamt enthält die noch nicht weiter reduzierte ungefilterte Datenmenge die Beschreibung von {{len(diagrams)}}  Diagrammen.  (Bemerkung: die in diesem Satz angegebene Zahl wurde wie zuvor in der Python Zelle berechnet.)  Die Keys  mit dem Zusatz „ID“  sind eindeutige Kennzeichen für die benannten Entitäten, in diesem Fall  ein Diagramm, dass den Namen *diaID* oder alternativ *altID*  trägt, sowie der Text *textID* aus einem Manuskripts *manID*, der immer eindeutig aus dem Werk eines Autors stammt und mit dem Kennzeichen *textID*  benannt ist.
 
 \begin{tabular}{lc{12cm}}
 'altID'&  alternative ID  des Diagramms\\
 'author'&  antiker Autor des Werks\\
 'biblio'&  die  Bezeichnung des Werks der Bibliothek, in dem sich heute befindet\\
 'diaAttr'&  eine Liste mit Merkmalen des Diagramms, die manuell erstellt wurde\\
 'diaID'& ID des  Diagramms\\
 'diaTyp'&  Typ des Diagramms nach der Konvention des Projekts\\
 'diaURL'&   Teil einer URL, mit der das Diagramm aus dem Repositorium der Edition Topoi aufgerufen werden kann.\\
 'foliopage'&  Die Folioseite, auf der  sich das Diagramm  befindet\\
 'manID'& ID  des Manuskripts, in dem der Text mit den Diagramm enthalten ist\\
 'manURL'&  Teil einer URL für den Aufruf des Faksimile  des Manuskripts\\
 'textID'&ID  des Textes eines Autors\\
 'textURL'&  Teil einer URL für den Aufruf des Faksimile ist des Texts\\
 'year& geschätzte Zeit für die Abschrift \\
 \end{tabular}

# Dataframes

## Erweiterung der Felder: 'centuries'

 im ersten Schritt erweitern wir den Datensatz um eine neue Spalte 'century', für die das Jahrhundert aus der Angabe des Jahres berechnet wird. Die Datierung der Manuskripte wurde oft aufgrund der paleographischen Eigenheiten der Schrift in ein Jahrhundert datiert, oft ist es auch für ein bestimmtes Skriptorium oder gar Schreiber zu datieren gewesen und damit zeitlich enger einzugrenzen gewesen, in wenigen Ausnahmen konnte ein genaues Jahr für die Abfassung einer Abschrift gegeben werden. In der neuen Spalte werden diese Angaben vergleichbar auf das ganze Jahrhundert reduziert.

In [ ]:
diagrams['century']=diagrams['year'].apply(lambda r: roda.from_roman(r))

In [ ]:
diagrams.head(1)

## Auswahl

Mit der Funktion reducedData(dataframe, keyValueList,debug=False)  lassen sich die Zeilen des Datensatzes auch solche reduzieren, deren Key  den String als Wert enthält. auf diese Weise kann schnell überprüft werden, ob bestimmte Sonderfälle oder Ausreißer der Daten vorkommen und im nächsten Schritt der Bereinigung des Datensatzes eliminiert werden sollen. Im ersten Fall wird überprüft, ob ein Jahrhundert entweder nicht angegeben ist und leer ist, oder ob der Wert mit dem Jahrhundert null angegeben ist.  In diesem Fall konnte  zur Datierung der Manuskripte keine zuverlässige Zeitangabe geschätzt werden.


In [ ]:
medDia.reducedData(diagrams,[["century",0]])

## Cleaning data
Leave out 
   - diagrams without corresponding diatyp
   - unclear year

In [ ]:
diagrams=diagrams[diagrams["diaTyp"] != 0.0]
len(diagrams)

In [ ]:
diagrams=diagrams[diagrams["diaTyp"] != '']
len(diagrams)

In [ ]:
diagrams=diagrams[diagrams["century"] != 0]
anzahlDia=len(diagrams)
anzahlDia

In [ ]:
diagrams.info()

## Mengen: Autoren, years and centuries

In [ ]:
setAutoren=set(pd.Series(diagrams["author"]))
setAutoren

In [ ]:
years=pd.Series(diagrams['year'])
setYears=set(years)
# setYears

In [ ]:
setTextID=set(pd.Series(diagrams["textID"]))
len(setTextID)

In [ ]:
setDiaTyp=set(pd.Series(diagrams["diaTyp"]))
setDiaTyp

In [ ]:

setCent=set(pd.Series(diagrams["century"]))
setCent

## Häufigkeit und Verteilung der Diagramme

Die Datenbank enthält {{anzahlDia}}  Diagramme eines fest zugeordneten Typs. Insgesamt verteilen sich die Diagramme über die vier verschiedenen Autoren in folgender Häufigkeit. Die Anzahl der Diagramme ist für Calcidius fast doppelt so groß wie für die drei übrigen Autoren.

In [ ]:
diagrams.groupby("author").size()

In [ ]:
diagrams.groupby(["author","century"]).size()

In [ ]:
pd.crosstab([diagrams["author"]],diagrams["century"],margins=True)

In [ ]:
pd.crosstab([diagrams["author"],diagrams["diaTyp"]],diagrams["century"],margins=True)

## Häufigkeit und Verteilung der Texte mit Diagrammen

Die Datenbank enthält {{anzahlTexte}}  Diagramme eines fest zugeordneten Typs. Insgesamt verteilen sich die Diagramme über die vier verschiedenen Autoren in folgender Häufigkeit. Die Anzahl der Diagramme ist für Calcidius fast doppelt so groß wie für die drei übrigen Autoren.

In [ ]:
texteVert=diagrams.groupby(["textID","century","author"]).size().to_frame(name='number dia').reset_index()
anzTexte=len(texteVert)

In [ ]:
texteVert

In [ ]:
texte=diagrams.groupby('textID').agg(lambda x: "|".join(x.sort_values().unique().tolist()))
texte.head(2)

In [ ]:
diagrams.groupby(["author","century","diaTyp"]).size()

## Counts

In [ ]:
diagrams["author"].value_counts()

In [ ]:
diagrams["diaTyp"].value_counts()

In [ ]:
diagrams["century"].value_counts()